In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime

In [2]:
driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
url = "https://le-meridien.marriott.com/hotel-directory/"
driver.get(url)
time.sleep(5)
# viewhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[3]/button')[0]
# viewhotels_button.click()
# northamhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[2]/pacsys-directory-listings/div/div[5]/a')[0]
# northamhotels_button.click()

In [3]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [4]:
stuff = soup.find_all('a', {'class', 'property'})
# stuff
hotel_list = []
for x in stuff:
    hotel = {}
    hotel['name'] = x.text.strip()
    hotel['url'] = x.attrs['href']
    hotel_list.append(hotel)
achotels_df = pd.DataFrame(hotel_list)
achotels_df

,name,url
0,Le Méridien Oran Hotel & Convention Centre,https://www.marriott.com/ORNMD
1,Le Méridien Cairo Airport,https://www.marriott.com/CAIAM
2,Le Méridien Heliopolis,https://www.marriott.com/CAIHM
3,Le Méridien Pyramids Hotel & Spa,https://www.marriott.com/CAIMD
4,Le Méridien Dahab Resort,https://www.marriott.com/SSHMD
...,...,...
110,Le Méridien Philadelphia,https://www.marriott.com/PHLMD
111,Le Méridien Dallas by the Galleria,https://www.marriott.com/DALMD
112,"Le Méridien Dallas, The Stoneleigh",https://www.marriott.com/DALML
113,Le Méridien Houston Downtown,https://www.marriott.com/HOUDM


In [5]:
%%time

hotel_list = []
counter = 0
for index, row in achotels_df.iterrows():
    name = row['name']
    url = row['url']
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        stuff = soup.find('div', {'class' : 'l-container l-header-container'})
#             hotel_list = []
        hotel = {}
        try:
            hotel['brand'] = 'Le Meredian'
        except:
            hotel['brand'] = 'Le Meredian'
        try:
            hotel['name'] = name
        except:
            hotel['name'] = name
        try:
            hotel['url'] = url
        except:
            hotel['url'] = url
        try:
            hotel['street'] = stuff.find_all('span')[2].text
        except:
            hotel['street'] = 'N/A'
        try:
            hotel['locality'] = stuff.find_all('span')[4].text
        except:
            hotel['locality'] = 'N/A'        
        try:
            hotel['state'] = stuff.find_all('span')[6].text
        except:
            hotel['state'] = 'N/A'            
        try:
            hotel['postalcode'] = stuff.find_all('span')[7].text
        except:
            hotel['postalcode'] = 'N/A'  
        try:
            hotel['coordinate1'] = stuff.find_all('span')[25].text
        except:
            hotel['coordinate1'] = 'N/A'            
        try:
            hotel['coordinate2'] = stuff.find_all('span')[26].text
        except:
            hotel['coordinate2'] = 'N/A'
        try:
            hotel['coordinate3'] = stuff.find_all('span')[27].text
        except:
            hotel['coordinate3'] = 'N/A'            
        try:
            hotel['coordinate4'] = stuff.find_all('span')[28].text
        except:
            hotel['coordinate4'] = 'N/A'            
        
        desc = soup.find_all('p', {'class' : 'l-margin-top-none l-overflow-hidden t-collapsed-height'})
        try:
            hotel['description'] = str(desc).replace('<p class="l-margin-top-none l-overflow-hidden t-collapsed-height">','').replace('</p>', '')
        except:
            hotel['description'] = 'N/A'
        
        overallratings = stuff = soup.find('div', {'class' : 'l-overall-rating-container'})
        try:
            hotel['overall_rating'] = ratings.find_all('div')[0].text.strip()[:4]
        except:
            hotel['overall_rating'] = 'N/A'
        
        specratings = stuff = soup.find('div', {'class' : 'l-secondary-ratings-container l-clear l-margin-bottom-quarter l-m-margin-bottom-none'})
        try:
            hotel['cleanliness_rating'] = specratings.find_all('div')[2].text.strip()[:4]
        except:
            hotel['cleanliness_rating'] = 'N/A'
        try:
            hotel['service_rating'] = specratings.find_all('div')[7].text.strip()[:4]
        except:
            hotel['service_rating'] = 'N/A'
            
        hotel_list.append(hotel)
        print(hotel)
        print()
        print(datetime.now())
        print('='*80)
        print('='*80)
        print()
#         driver.close()
#         driver.switch_to.window(original_window)
#         time.sleep(1)

        counter += 1
        print(f'Collected {counter} hotels.')
    except:
        hotel = {}
        hotel['name'] = name
        hotel['url'] = url
        hotel_list.append(hotel)
        print(f"Number {x} didn't work at {datetime.now()}")
        continue
hotel_df = pd.DataFrame(hotel_list)
hotel_df   

{'brand': 'Le Meredian', 'name': 'Le Méridien Oran Hotel & Convention Centre', 'url': 'https://www.marriott.com/ORNMD', 'street': 'Les Genets, Chemin de Wilaya, Route 75', 'locality': 'Oran', 'state': '', 'postalcode': '', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/ornmd-le-meridien-oran-hotel-and-convention-centre/?directPageRequest=true', 'coordinate2': '35.726857', 'coordinate3': '-0.592296', 'coordinate4': 'N/A', 'description': '[ Perched on a cliff, revealing views of the Mediterranean Sea, Le Méridien Oran Hotel &amp; Convention Centre is a call to discovery. Equipped with a state-of-the-art conference center, with a total of 296 rooms and suites, 28 meeting rooms and an auditorium of 3000 seats. Your next lecture, social or professional event is only limited by your imagination. Featuring balconies and floor-to-ceiling windows, the sleek and modern hotel rooms offer free Wi-Fi and flat-screen TVs among their various amenities; upgraded rooms offer stunning sea v

{'brand': 'Le Meredian', 'name': 'Le Méridien Ile Maurice', 'url': 'https://www.marriott.com/MRUMD', 'street': 'Village Hall Lane', 'locality': 'Pointe aux Piments', 'state': '', 'postalcode': '', 'coordinate1': ' View Rates ', 'coordinate2': '', 'coordinate3': ' View Rates ', 'coordinate4': 'https://www.marriott.com/hotels/maps/travel/mrumd-le-meridien-ile-maurice/?directPageRequest=true', 'description': '[ Overlooking the sparkling waters of the Indian Ocean, Le Méridien Ile Maurice is the perfect resort for a beach escape in Mauritius. Our luxury beachfront hotel in Pointe Aux Piments stands on the edge of the largest natural beach in Mauritius with sweeping ocean views and sits minutes away from an array of attractions, including Grand Bay restaurants and bars, the Central Market and the Mauritius Aquarium. Attractive to families, small groups or solo travelers, our hotel rooms and suites are designed for comfort. Entertain in one of our well-appointed suites, or escape to the ‘adu

{'brand': 'Le Meredian', 'name': 'Le Méridien Thimphu', 'url': 'https://www.marriott.com/PBHMD', 'street': 'Chorten Lam, P.O. Box 01286', 'locality': 'Thimphu', 'state': '', 'postalcode': '', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/pbhmd-le-meridien-thimphu/?directPageRequest=true', 'coordinate3': '27.557989', 'coordinate4': '89.560547', 'description': "[ Choose Le Méridien Thimphu for an unforgettable experience. Located at the heart of Bhutan's capital city, we are just steps away from shops and entertainment, and conveniently accessible to Memorial Chorten, Tashichho Dzong and other celebrated attractions. Our 78 elegant guest rooms and suites boast authentic Bhutanese-style furnishings and stellar amenities like flat-screen TV, free high speed wi-fi, marble bathroom, an ergonomic workspace and uncompromising views of the local hillsides and the capital city .Be inspired and energised by the global flavours and creative presentations of 

{'brand': 'Le Meredian', 'name': 'Le Méridien Shimei Bay Beach Resort & Spa', 'url': 'https://www.marriott.com/SYXMD', 'street': 'Shimei Bay, Shimei Bay', 'locality': 'Wanning', 'state': '', 'postalcode': '571533', 'coordinate1': '110.285041', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ Allow Le Méridien Shimei Bay Beach Resort &amp; Spa to raise the bar for a 5-star hotel visit. We're ideally situated on Hainan Island, China's breathtaking Shimei Bay, southeast of Wanning. Our resort is set on 30 acres of lush, immaculately maintained landscape, steps away from the sparkling South China Sea and close to many notable attractions. Each of our spacious villas features exquisite design touches, a balcony with outstanding views of Hainan and a wealth of premium perks. You'll be thrilled with our hotel's 5-star amenities, which include private beach access, six first-class restaurant and bar options, a well-stocked fitness center, a tranquil spa, two 

{'brand': 'Le Meredian', 'name': 'Le Méridien Shenyang, Heping', 'url': 'https://www.marriott.com/SHEMD', 'street': "No. 45, Bei'er Road, Heping District", 'locality': 'Shenyang', 'state': '', 'postalcode': '110001', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/shemd-le-meridien-shenyang-heping/?directPageRequest=true', 'coordinate3': '41.79389', 'coordinate4': '123.407104', 'description': '[ Located in the Centre of Shenyang, the Le Méridien Shenyang, Heping is the perfect base to explore the city as it is situated only steps away from Taiyuan Shopping Street and is easily accessible from Shenyang Train Station. Featuring 296 elegant guest rooms and suites with high-speed broadband and Wi-Fi access, signature LM bed and 55 inch flat-panel TV, with a separate, inviting, deep-soaking tub and rainforest shower. The room also offers a wide variety of extraordinary service such as Le Meridien Hub, two signature restaurants and Lobby Bar. When stayin

...

{'brand': 'Le Meredian', 'name': 'Le Méridien Delfina Santa Monica', 'url': 'https://www.marriott.com/LAXDM', 'street': '530 Pico Boulevard', 'locality': 'Santa Monica', 'state': 'California', 'postalcode': '90405', 'coordinate1': 'https://www.marriott.com/hotels/maps/travel/laxdm-le-meridien-delfina-santa-monica/?directPageRequest=true', 'coordinate2': '34.010535', 'coordinate3': '-118.485711', 'coordinate4': 'N/A', 'description': '[ Stimulate your passion for discovery at Le Méridien Santa Monica, a boutique hotel that invites you to explore Southern California in style. Four blocks from the beach and within walking distance from the Santa Monica Pier, take in the opportunities to connect, disconnect or simply savor the moment. Your arrival is met with thoughtful hints of the hospitality to follow: free valet parking and two glasses of champagne. Our house car awaits to take you anywhere within two miles. Or perhaps a free beach bike is more your speed? A state-of-the-art 24/7 fitnes

{'brand': 'Le Meredian', 'name': 'Le Méridien Chicago - Oakbrook Center', 'url': 'https://www.marriott.com/CHIMB', 'street': '2100 Spring Road', 'locality': 'Oak Brook', 'state': 'Illinois', 'postalcode': '60523', 'coordinate1': 'Reviews', 'coordinate2': ' View Rates ', 'coordinate3': '', 'coordinate4': ' View Rates ', 'description': "[ Ripe with contemporary calm and artful sophistication, Le Méridien Chicago - Oakbrook Center is situated adjacent to its eponymous shopping destination in Oak Brook, Illinois. From this superb vantage point, hotel guests can access Oak Brook's most prominent attractions with ease, including Drury Lane Theatre, the Elmhurst Art Museum and downtown Chicago. Providing an inspiring setting to work and play, our guest rooms and suites feature contemporary artwork, luxurious bedding and state-of-the-art technology. Join us at El Tapeo, where modern Spanish cuisine and breathtaking views of Oak Brook take center stage. For a more casual dining experience, head

{'brand': 'Le Meredian', 'name': 'New: Le Méridien St. Louis Clayton', 'url': 'https://www.marriott.com/STLMY', 'street': '7730 Bonhomme Avenue', 'locality': 'St. Louis', 'state': 'Missouri', 'postalcode': '63105', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ Le Meridien St. Louis Clayton is where the creative minded, the culture seekers and those who live to savor the moment unlock the destination of St. Louis. With 268 thoughtfully designed hotel rooms and 17,000 square feet of stylish event space, we guide our guests through the nuances of what makes this city great. Located in the epicenter of the chic Clayton business district, minutes from Forest Park and downtown Saint Louis, this Clayton hotel celebrates art through an effortless blend of simplicity and comfort with style and detail. Delight in the nostalgia of form and function in our nod to Mid-Century Modern design while remaining fearless to indulge in the new and

{'brand': 'Le Meredian', 'name': 'Le Méridien Dallas, The Stoneleigh', 'url': 'https://www.marriott.com/DALML', 'street': '2927 Maple Avenue', 'locality': 'Dallas', 'state': 'Texas', 'postalcode': '75201', 'coordinate1': '-96.807373', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ Classic and modern elegance are juxtaposed at the chic uptown location of Le Méridien Dallas, The Stoneleigh. This historic 1923 Art Deco building is steps from businesses, museums, boutique shopping, cutting-edge cuisine and stimulating nightlife. Perle on Maple Restaurant serves innovative dishes that showcase inspiring cuisine with a distinctive Texas twist. Dallas meetings and events are hosted with flair in 6,000 square feet of Dorothy Draper-designed space. In each of our 176 guest rooms and suites, warm cinnabar and charcoal color palettes blend modern amenities with custom mahogany and marble-topped furniture. Bathrooms feature sleek black-and-white Calcutta marble

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Le Meredian,Le Méridien Oran Hotel & Convention Centre,https://www.marriott.com/ORNMD,"Les Genets, Chemin de Wilaya, Route 75",Oran,,,https://www.marriott.com/hotels/maps/travel/or...,35.726857,-0.592296,N/A,"[ Perched on a cliff, revealing views of the M...",N/A,4.1,3.9
1,Le Meredian,Le Méridien Cairo Airport,https://www.marriott.com/CAIAM,"Cairo International Airport, Terminal 3",Cairo,,,Reviews,View Rates,,View Rates,[ Awaken your sense of wonder when you stay at...,N/A,N/A,N/A
2,Le Meredian,Le Méridien Heliopolis,https://www.marriott.com/CAIHM,"51 El Orouba Street, Heliopolis",Cairo,,,Reviews,https://www.marriott.com/hotels/maps/travel/ca...,30.091431,31.336259,"[ No matter your reason for visiting Egypt, Le...",N/A,4.0,3.9
3,Le Meredian,Le Méridien Pyramids Hotel & Spa,https://www.marriott.com/CAIMD,"El Remaya Square - Pyramids, PO Box 25 Pyramids",Cairo,,12561,View Rates,,View Rates,https://www.marriott.com/hotels/maps/travel/ca...,"[ Le Méridien Pyramids Hotel &amp; Spa, nestle...",N/A,N/A,N/A
4,Le Meredian,Le Méridien Dahab Resort,https://www.marriott.com/SSHMD,"PO Box 2, South Sinai, Red Sea",Dahab,,,Reviews,https://www.marriott.com/hotels/maps/travel/ss...,28.469899,34.478188,[ Discover shimmering golden sands and swaying...,N/A,4.5,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,Le Meredian,Le Méridien Philadelphia,https://www.marriott.com/PHLMD,1421 Arch Street,Philadelphia,Pennsylvania,19102,https://www.marriott.com/hotels/maps/travel/ph...,39.954444,-75.163538,N/A,"[ Our hotel is housed in a century old, 10-sto...",N/A,4.3,4.2
111,Le Meredian,Le Méridien Dallas by the Galleria,https://www.marriott.com/DALMD,13402 Noel Road,Dallas,Texas,75240,-96.818098,N/A,N/A,N/A,[ From the chic mid-century modern decor adorn...,N/A,4.3,4.2
112,Le Meredian,"Le Méridien Dallas, The Stoneleigh",https://www.marriott.com/DALML,2927 Maple Avenue,Dallas,Texas,75201,-96.807373,N/A,N/A,N/A,[ Classic and modern elegance are juxtaposed a...,N/A,4.4,4.1
113,Le Meredian,Le Méridien Houston Downtown,https://www.marriott.com/HOUDM,1121 Walker Street,Houston,Texas,77002,29.756986,-95.362711,N/A,N/A,[ Explore the seamless combination of contempo...,N/A,4.6,4.2


In [6]:
hotel_df = hotel_df.drop_duplicates(subset='name')

In [7]:
hotel_df

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Le Meredian,Le Méridien Oran Hotel & Convention Centre,https://www.marriott.com/ORNMD,"Les Genets, Chemin de Wilaya, Route 75",Oran,,,https://www.marriott.com/hotels/maps/travel/or...,35.726857,-0.592296,N/A,"[ Perched on a cliff, revealing views of the M...",N/A,4.1,3.9
1,Le Meredian,Le Méridien Cairo Airport,https://www.marriott.com/CAIAM,"Cairo International Airport, Terminal 3",Cairo,,,Reviews,View Rates,,View Rates,[ Awaken your sense of wonder when you stay at...,N/A,N/A,N/A
2,Le Meredian,Le Méridien Heliopolis,https://www.marriott.com/CAIHM,"51 El Orouba Street, Heliopolis",Cairo,,,Reviews,https://www.marriott.com/hotels/maps/travel/ca...,30.091431,31.336259,"[ No matter your reason for visiting Egypt, Le...",N/A,4.0,3.9
3,Le Meredian,Le Méridien Pyramids Hotel & Spa,https://www.marriott.com/CAIMD,"El Remaya Square - Pyramids, PO Box 25 Pyramids",Cairo,,12561,View Rates,,View Rates,https://www.marriott.com/hotels/maps/travel/ca...,"[ Le Méridien Pyramids Hotel &amp; Spa, nestle...",N/A,N/A,N/A
4,Le Meredian,Le Méridien Dahab Resort,https://www.marriott.com/SSHMD,"PO Box 2, South Sinai, Red Sea",Dahab,,,Reviews,https://www.marriott.com/hotels/maps/travel/ss...,28.469899,34.478188,[ Discover shimmering golden sands and swaying...,N/A,4.5,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,Le Meredian,Le Méridien Philadelphia,https://www.marriott.com/PHLMD,1421 Arch Street,Philadelphia,Pennsylvania,19102,https://www.marriott.com/hotels/maps/travel/ph...,39.954444,-75.163538,N/A,"[ Our hotel is housed in a century old, 10-sto...",N/A,4.3,4.2
111,Le Meredian,Le Méridien Dallas by the Galleria,https://www.marriott.com/DALMD,13402 Noel Road,Dallas,Texas,75240,-96.818098,N/A,N/A,N/A,[ From the chic mid-century modern decor adorn...,N/A,4.3,4.2
112,Le Meredian,"Le Méridien Dallas, The Stoneleigh",https://www.marriott.com/DALML,2927 Maple Avenue,Dallas,Texas,75201,-96.807373,N/A,N/A,N/A,[ Classic and modern elegance are juxtaposed a...,N/A,4.4,4.1
113,Le Meredian,Le Méridien Houston Downtown,https://www.marriott.com/HOUDM,1121 Walker Street,Houston,Texas,77002,29.756986,-95.362711,N/A,N/A,[ Explore the seamless combination of contempo...,N/A,4.6,4.2


In [8]:
hotel_df.to_csv('lemeridian_description.csv')